In [6]:
import os

os.chdir("C:/Users/KOBIC/Documents/latentgee/")
!pip install -e .

Obtaining file:///C:/Users/KOBIC/Documents/latentgee
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build editable did not run successfully.
  │ exit code: 1
  ╰─> [63 lines of output]
      C:\Users\KOBIC\AppData\Local\Temp\pip-build-env-lrwyai5j\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:75: _MissingDynamic: `requires-python` defined outside of `pyproject.toml` is ignored.
      !!
      
              ********************************************************************************
              The following seems to be defined outside of `pyproject.toml`:
      
              `requires-python = '>=3.8'`
      
              According to the spec (see the link below), however, setuptools CANNOT
              consider this value unless `requires-python` is listed as `dynamic`.
      
              https://packaging.python.org/en/latest/specifications/pyproject-toml/#declaring-project-metadata-the-project-table
      
              To prevent this problem, you can list `requires-python` und

In [26]:
import os
import numpy as np
import pandas as pd
import sklearn
import optuna
import torch
import torch.nn as nn
import random
from typing import Tuple, Dict

from hdbscan import HDBSCAN
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler



In [25]:
file_path = "F:/졸업 후 연구/LatentGEE/Data"

dat = pd.read_csv(os.path.join(file_path, "insight.merged_otus.txt"), sep="\t", encoding = "utf-8")
dat_meta = pd.read_excel(os.path.join(file_path, "SupplementaryMaterial.xlsx"), header = 1, usecols = "B:F")
dat_seqID = dat_meta["SeqID"].astype(str).to_list()
dat_T = dat[dat_seqID].T
dat_T.reset_index(inplace = True)
dat_T.columns = dat_T.iloc[0].astype(str)
dat_T = dat_T.iloc[1:]
dat_T.columns.values[0] = "SeqID"
dat_merged = pd.merge(dat_T, dat_meta[["SeqID", "Study.1", "Study", "hivstatus", "Age"]], on = "SeqID", how = "inner")

X_tensor = dat_T.iloc[:, 1:].apply(pd.to_numeric, errors="coerce")
X_tensor = torch.tensor(X_tensor.values, dtype = torch.float32)

In [5]:
dat_meta.head()

,Study,Study.1,SeqID,hivstatus,Age
0,Dillon,Dillion.2014,SRR1159399,0,27.0
1,Dillon,Dillion.2014,SRR1159402,1,48.0
2,Dillon,Dillion.2014,SRR1159405,1,25.0
3,Dillon,Dillion.2014,SRR1159408,0,29.0
4,Dillon,Dillion.2014,SRR1159411,1,58.0


In [6]:
dat.head()

,Resphera Insight (Raw Counts),4EP1,4EP10,4EP12,4EP15,4EP16,4EP17,4EP19,4EP20,4EP21,...,rgn.8.9,rgn.9.10,rgn.9.11,rgn.9.12,rgn.9.13,rgn.9.2,rgn.9.3,rgn.9.6,rgn.9.9,taxonomy
0,Abiotrophia_defectiva,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lac...
1,Acetatifactor_muris:Blautia_coccoides:Blautia_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,k__Bacteria; p__Firmicutes; c__Clostridia; o__...
2,Acetitomaculum_ruminis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,k__Bacteria; p__Firmicutes; c__Clostridia; o__...
3,Acetobacter_fabarum:Acetobacter_ghanensis:Acet...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,k__Bacteria; p__Proteobacteria; c__Alphaproteo...
4,Acetobacterium_carbinolicum:Acetobacterium_fim...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,k__Bacteria; p__Firmicutes; c__Clostridia; o__...


In [7]:
print("torch ==", getattr(torch, "__version__", None))
print("numpy ==", getattr(np, "__version__", None))
print("scikit-learn ==", getattr(sklearn, "__version__", None))
print("optuna ==", getattr(optuna, "__version__", None))
print("hdbscan ==", getattr(HDBSCAN, "__version__", None))

torch == 2.9.0.dev20250827+cu128
numpy == 2.1.2
scikit-learn == 1.7.1
optuna == 4.5.0
hdbscan == None


In [22]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# new
def save_cache(path, X_tensor, input_dim, cutoff):
    torch.save(
        {"X_tensor": X_tensor, "input_dim": input_dim, "cutoff": cutoff},
        path
    )

# new 
def subset_by_cutoff(X, cutoff):
    n_samples = X.shape[0]
    
     # prevalence 계산: 각 OTU의 nonzero 비율
    prevalence = (X > 0).sum(axis=0) / n_samples
    filtered_df = X.loc[:, prevalence >= cutoff]
    
    return torch.from_numpy(filtered_df.values).float()

# 메모리 캐시
_MEMORY_CACHE: Dict[float, Tuple[torch.Tensor, int]] = {}

# 디스크 캐시

cache_dir = "C:/Users/KOBIC/Documents/latentgee/experiments"
os.makedirs(cache_dir, exist_ok=True)
#cache_path = os.path.join(cache_dir, f"X_tensor_cutoff_{cutoff_key}.pt")
   
def get_X_tensor(
    raw_data, 
    cutoff : float, 
    cache_dir :str = "./tensor_cache"
    ) -> Tuple[torch.Tensor, int]:
    """
    cutoff별 X_tensor를 CPU 텐서로 반환하고 input_dim을 리턴.
    동작 순서:
        1) 메모리 캐시 확인
        2) 디스크 캐시 확인
        3) 없으면 raw 데이터로부터 전처리 후 저장
    
    Returns
    ---------
    X_tensor : torch.Tensor(cpu)
    input_dim : int
    """
    global _MEMORY_CACHE
    cutoff_key = int(cutoff * 10000) # Python의 float는 IEEE 754 배정밀도라서 많은 소수가 정확히 표현되지 않으므로 방어적으로 정수 스케일링 처리.
    
    # --------------------------------------
    # 1. 메모리 캐시 확인
    # --------------------------------------
    if cutoff_key in _MEMORY_CACHE:
        return _MEMORY_CACHE[cutoff_key]
    
    # --------------------------------------
    # 2. 디스크 캐시 확인
    # -------------------------------------- 
    if os.path.exists(cache_path):
        data = torch.load(cache_path, map_location="cpu")
        X_tensor = data["X_tensor"]
        input_dim = data["input_dim"]
        
        # 메모리 캐시에 저장
        _MEMORY_CACHE[cutoff_key] = (X_tensor, input_dim)
        return X_tensor, input_dim
    
    # --------------------------------------
    # 3. 전처리 수행
    # --------------------------------------    
    X_tensor = subset_by_cutoff(raw_data, cutoff)
    
    if not isinstance(X_tensor, torch.Tensor):
        raise TypeError("subset_by_cutoff must return torch.Tensor")
    
    X_tensor = X_tensor.cpu() 
    input_dim = X_tensor.shape[1] if X_tensor.ndim > 1 else 1

    # 메모리 캐시 저장
    _MEMORY_CACHE[cutoff_key] = (X_tensor, input_dim)

    save_cache(cache_path, X_tensor, input_dim)
    return X_tensor, input_dim



def make_objective_for_X(
    X, y_batch, y_bio,
    ss_model, ss_training, ss_cluster, ss_eval
):
    def objective(trial: optuna.Trial):
        
        # -------- Model search space --------
        latent_dim = trial.suggest_int("latent_dim", ss_model["latent_dim"][0], ss_model["latent_dim"][1])
        n_layers = trial.suggest_int("n_layers", ss_model["n_layers"][0], ss_model["n_layers"][1])
        base_dim = trial.suggest_categorical("base_dim", ss_model["base_dim"])
        dropout_rate = trial.suggest_float("dropout_rate", ss_model["dropout_rate"][0], ss_model["dropout_rate"][1])
        activation = trial.suggest_categorical("activation", ss_model["activation"])
        strategy = trial.suggest_categorical("strategy", ss_cluster["strategy"])
        
        # -------- Training search space --------
        epochs = trial.suggest_categorical("epochs", ss_train["epochs"])
        batch_size = trial.suggest_categorical("batch_size", ss_train["batch_size"])
        lr_low, lr_high = ss_train["learning_rate"]["loguniform"]
        lr = trial.suggest_float("lr", lr_low, lr_high, log=True)
        optimizer = ss_train["training"]["optimizer"]
        amp = ss_train["training"]["amp"]
        grad_clip_norm = ss_train["training"]["grad_clip_norm"]
        scheduler = ss_train["training"]["scheduler"]
        warmup_epochs = ss_train["training"]["warmup_epochs"]
        
        # -------- Config objects --------
        model_cfg = ModelConfig(
            input = input_dim,
            latent_dim = latent_dim,
            n_layers = n_layers,
            base_dim = base_dim,
            strategy = strategy,
            dropout = dropout_rate,
            activation = activation
        )
        
        train_cfg = TrainConfig(
            device = "cuda" if torch.cuda.is_available() else "cpu",
            epochs = epochs,        
            batch_size = batch_size,
            lr = lr,
            optimizer = optimizer,
            amp = amp,
            grad_clip_norm = grad_clip_norm,           # 그래디언트 클리핑 max_norm
            scheduler = scheduler,
            warmup_epochs = warmup_epochs
        )
        eval_cfg = EvalConfig(
            hdb_min_cluster_size=ss_cluster["min_cluster_size"][0],
            hdb_metric=ss_eval["silhouette_metric"][0]
            )

        # -------- Run pipeline --------
        pipeline = LatentGEEPipeline(
            X=X,
            model_cfg=model_cfg,
            train_cfg=train_cfg,
            eval_cfg=eval_cfg,
        )

        pipeline.fit()

        res = pipeline.evaluate_pipeline(
            original_batch_labels=y_batch,
            original_bio_labels=y_bio
        )
        score = res["after_batch"]["silhouette"] - res["after_bio"]["silhouette"]
        return score
    
    return objective
        
    

In [35]:
os.chdir("C:/Users/KOBIC/Documents/latentgee")
!pip install -e .

Obtaining file:///C:/Users/KOBIC/Documents/latentgee
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build editable did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      C:\Users\KOBIC\AppData\Local\Temp\pip-build-env-mjzmgjf8\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:75: _MissingDynamic: `requires-python` defined outside of `pyproject.toml` is ignored.
      !!
      
              ********************************************************************************
              The following seems to be defined outside of `pyproject.toml`:
      
              `requires-python = '>=3.8'`
      
              According to the spec (see the link below), however, setuptools CANNOT
              consider this value unless `requires-python` is listed as `dynamic`.
      
              https://packaging.python.org/en/latest/specifications/pyproject-toml/#declaring-project-metadata-the-project-table
      
              To prevent this problem, you can list `requires-python` und

In [ ]:
import sys
sys.path.append("C:/Users/KOBIC/Documents/latentgee")

from latentgee.config import load_cfg, ModelConfig, TrainConfig, EvalConfig
from latentgee.pipeline import LatentGEEPipeline
from latentgee.core import train_vae

ModuleNotFoundError: No module named 'latentgee'

In [30]:
set_seed(42)

cfg = load_cfg("C:/Users/KOBIC/Documents/latentgee/experiments/config.yaml")

ss_model = cfg["search_space"]["model"]
ss_train = cfg["search_space"]["train"]
ss_cluster = cfg["search_space"]["clustering"]
ss_eval = cfg["search_space"]["eval"]

cutoff = cfg["data"]["zero_prevalence_cutoff"][0]
X_tensor, input_dim = get_X_tensor(dat, cutoff)

X = X_tensor.numpy().astype("float32")

y_batch = dat_meta["Study"].values
y_bio = dat_meta[["hivstatus", "Age"]].values

NameError: name 'load_cfg' is not defined

In [ ]:
set_seed(42)

cfg = load_cfg("C:/Users/KOBIC/Documents/latentgee/experiments/config.yaml")

ss_model = cfg["search_space"]["model"]
ss_train = cfg["search_space"]["train"]
ss_cluster = cfg["search_space"]["clustering"]
ss_eval = cfg["search_space"]["eval"]

cutoff = cfg["data"]["zero_prevalence_cutoff"][0]
X_tensor, input_dim = get_X_tensor(dat, cutoff)

X = X_tensor.numpy().astype("float32")

y_batch = dat_meta["Study"].values
y_bio = dat_meta[["hivstatus", "Age"]].values

# ============================================================
# 4️⃣ Define Optuna Objective
# ============================================================

cutoff_values = np.arange(0.00, 0.105, 0.005)

results_summary = []

for cutoff in cutoff_values:
    print(f"=== Running cutoff {cutoff:.3f} ===")
    
    
    X_tensor, input_dim = get_X_tensor(cutoff, cache_dir="C:/Uers/KOBIC/Documents/latentgee/X_cache_files")
    X = X_tensor.numpy().astype("float32")
    
    objective = make_objective_for_X(
       X =  X_tensor,
       y_batch = y_batch,
       y_bio = y_bio,
       
       ss_model = cfg["search_space"]["model"],
       ss_train = cfg["search_space"]["training"],
       ss_cluster = cfg["search_space"]["clustering"],
       ss_eval = cfg["search_space"]["evaluation"]
       )
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    
    best_params = study.best_trial.params
    
    model_cfg, train_cfg, eval_cfg = build_configs_from_best_params(
        best_params, input_dim, cfg
    )
    
    pipeline = LatentGEEPipeline(X, model_cfg, train_cfg, eval_cfg)
    pipeline.fit()
    
    metrics = pipeline.evaluate_pipeline(
        original_batch_labels = y_batch,
        original_bio_labels = y_bio
    )
    
    results_summary.append({
        "cutoff": cutoff,
        "best_params": best_params,
        "metrics": metrics,
        "study": study
    })
    

best_record = select_best_across_cutoffs(results_summary)

print("BEST_cutoff:", best_record["cutoff"])
print("BEST metrics:", best_record["metrics"])    
    
    
    
    
    
    





NameError: name 'set_seed' is not defined